## tf.data使用　参考<https://blog.csdn.net/qq_37747262/article/details/81912438>

In [4]:
import tensorflow as tf 
import numpy as np

### 版本１. API接口过时。　<https://zhuanlan.zhihu.com/p/27238630>

In [ ]:
with tf.Session() as sess:
    filename = ['a.png', 'b.png', 'c.png']
    # string_input_producer：　输入文件名列表，　返回一个文件名队列
    filename_queue = tf.train.string_input_producer(filename, shuffle=False, num_epochs=5)
    
    # reader从文件名队列中读数据。对应的方法是reader.read
    reader = tf.WholeFileReader()
    key, value = reader.read(filename_queue) # ｋｅｙ是输入队列的名称，ｖａｌｕｅ是图片值 二进制
    
    # tf.train.string_input_producer定义了一个epoch变量，要对它进行初始化
    tf.local_variables_initializer().run()
    
    # 使用start_queue_runners之后，才会开始填充队列， key和ｖａｌｕe才可以运行
    threads = tf.train.start_queue_runners(sess=sess)
    
    i = 0
    while True:
        i += 1
        # 获取图片数据并保存
        image_data = sess.run(value)
        image_key  = sess.run(key)
        
        print("data size: {}".format(len(image_data)))
        print("data key:  {}".format(image_key))
#         with open('read/test_%d.png' % i, 'wb') as f:
#             f.write(image_data)

- tf.train.string_input_producer() --> tf.data.Dataset.from_tensor_slices  
- tf.WholeFileReader（）--> tf.data.Dataset.map()  
- tf.train.start_queue_runners()   --> tf.data

### Dataset API. https://zhuanlan.zhihu.com/p/30751039

#### from_tensor_slices 创造一个ｄａｔａｓｅｔ

In [17]:
dataset = tf.data.Dataset.from_tensor_slices(np.array([1.0, 2.0, 3.0, 4.0, 5.0]))

In [7]:
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()    # 这是一个tensor
with tf.Session() as sess:
    try:
        while True:
            print(sess.run(one_element))
    except tf.errors.OutOfRangeError: # 捕捉溢出异常
        print("end!")

1.0
2.0
3.0
4.0
5.0
end!


切片功能，　dataset中有５个元素，　每个元素形状(２，)

In [8]:
dataset = tf.data.Dataset.from_tensor_slices(np.random.uniform(size=(5,2)))

一个元素可以是元组，也可以是词典。  
字典：　分别切片“ａ", "b"中的数值，　dataset元素类似于{"a":1.0, "b":[0.9, 0.1]}

In [9]:
dataset = tf.data.Dataset.from_tensor_slices(
    {
        "label": np.array([0, 1, 2, 2, 1]),
        "b": np.random.uniform(size=(5,784))
    }
)

#### Transformation 变换元素

- map

In [12]:
dataset = tf.data.Dataset.from_tensor_slices(np.array([1.0, 2.0, 3.0, 4.0, 5.0]))
dataset = dataset.map(lambda x: x + 1, num_parallel_calls = 10) # 2.0, 3.0, 4.0, 5.0, 6.0

- batch

In [13]:
dataset = dataset.batch(32)

- shuffle

In [15]:
dataset = dataset.shuffle(buffer_size=10000)

- repeat 重复序列，可用于生成多个epoch，直接调用repeat()生成无限序列，不会抛出tf.errors.OutOfRangeError异常

In [16]:
dataset = dataset.repeat(5)

#### example  
读取Image及label, ｓｈｕｆｆｌｅ, batch_size=32, ｅｐｏｃｈ＝１０

In [ ]:
ImgHeight = 720
ImgWidth  = 1280
ShuffleSize = 1000

In [18]:
def _parse_function(filename, label):
    image_string = tf.read_file(filename) # 返回ｓｔｒｉｎｇ类型的tensor
    image_decoded = tf.image.decode_jpeg(image_string)
    image_resized = tf.image.resize_image(image_decoded, [ImgHeight, ImgWidth])
    return image_resized, label

filenames = tf.constant(["/var/data/image1.jpg", "/va/data/image2.jpg", ...])
ｌａｂｅｌｓ = tf.constant([0, 1, 2, 3, 5, ...])

# 元素是tuple, (ｆｉｌｅname, label)
dataset = tf.data.Dataset.from_tensor_slices((filenames,labels))

# (image_resized, label)
dataset = dataset.map(_parse_function)

# (image_resized_batch, label_batch)
# image_resized_batch: (32,720,1280,3)
# label_batch (32, )
dataset = dataset.shuffle(buffersize=ShuffleSize).batch(32).repeat(10)  
    

TypeError: Failed to convert object of type <class 'list'> to Tensor. Contents: ['/var/data/image1.jpg', '/va/data/image2.jpg', Ellipsis]. Consider casting elements to a supported type.

#### More Dataset  
tf.data.Data.from_tensor_slices
- tf.data.TextLineDataset()
- tf.data.FixedLengthRecordDataset()
- tf.data.TFRecordDataset()　！！！

#### More Iterator  ！！！！　对于大型数据集如何进行分步加载
dataset.make_one_shot_iterator()
- initializable iterator  
- reinitializable iterator
- feedable iterator

#### initializable iterator
initializable iterator必须要在使用前通过sess.run()来初始化。使用initializable iterator，可以将placeholder代入Iterator中，这可以方便我们通过参数快速定义新的Iterator。

In [20]:
limit = tf.placeholder(dtype=tf.int32, shape=[])

dataset = tf.data.Dataset.from_tensor_slices(tf.range(start=0, limit=limit))

iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={limit: 10})
    for i in range(10):
        value = sess.run(next_element)
        assert i == value

读入较大数组。　或许可以每次将一个ｂａｔｃｈ的数据进行传递，拿到一个ｂａｔｃｈ的数据后，放入迭代器中。

In [ ]:
ｗｉｔh np.load("/var/data/training_data.npy") as data:
    features = data["features"]
    labels   = data["labels"]
    
features_placeholder = tf.placeholder(features.dtype, features.shape)
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)

dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
iterator = dataset.make_initializable_iterator()
sess.run(iterator.initializer, feed_dict={features_placeholder: features,
                                          labels_placeholder: labels})